In [1]:
import csv
from time import time
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
class TWITTER():
    def __init__(self, directory):
        
        # if two users are following each other, counts are increased by 0.5, otherwise 1
        self.following = sc.textFile(directory) \
                                .map(lambda x: (x.split(" "*4)[0], x.split(" "*4)[1])) \
                                .map(lambda x: (sorted(x)[0]+sorted(x)[1],x[0])) \
                                .groupByKey() \
                                .flatMap(lambda x: [(list(x[1])[i],0.5) for i in range(len(x[1]))] if len(x[1])> 1 
                                         else [(list(x[1])[0], 1)]) \
                                .combineByKey((lambda x: x),
                                              # using combiner to minimize data shuffled across servers 
                                             (lambda x,y: x + y),
                                             (lambda x,y: x + y)).persist()
        
        
        self.followers = sc.textFile(directory) \
                                .map(lambda x: (x.split(" "*4)[0], x.split(" "*4)[1])) \
                                .map(lambda x: (sorted(x)[0]+sorted(x)[1],x[1])) \
                                .groupByKey() \
                                .flatMap(lambda x: [(list(x[1])[i],0.5) for i in range(len(x[1]))] if len(x[1])> 1 
                                         else [(list(x[1])[0], 1)]) \
                                .combineByKey((lambda x: x),
                                              # using combiner to minimize data shuffled across servers 
                                             (lambda x,y: x + y),
                                             (lambda x,y: x + y))
        
    # Calculate each user’s score (N*M)
    def user_score(self):
        s = self.following.join(self.followers).mapValues(lambda x: x[0] * x[1])
        return s
    
    # Save results as CSV 
    def save_file(self, file_name):
        with open(file_name, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(self.user_score().collect())

In [3]:
data = TWITTER(r"C:\Users\taylankabbani2019\Downloads\Connectivity_Score\*")

print(data.user_score().take(5))

# data.save_file('Connectivity_Scores')

[('user624', 309559.25), ('user8297', 186441.75), ('user3525', 365295.0), ('user6101', 362428.75), ('user7431', 261100.75)]
